I can organize this notebook more later

In [1]:
import numpy as np

In [2]:
xy = np.load("openface_and_labels_by_frame.npz")
X, y = xy["x"], xy["y"]

In [3]:
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (604901, 713)
y shape: (604901, 7)


In [4]:
print(X[1].shape)

(713,)


We can decide later exactly how much of the data ought to be used, but for the sake of testing things for now, I've truncated the dataset to something more manageable from a time perspective (sample a number (n_samples * m_percent) of random indices and extract those indices from the features/targets). I can perform a larger-in-scale training session over break on my GPU desktop (hopefully without interruption due to hardware limitations) once we're satisfied with the model parameterization

In [5]:
def truncate_data(X, y, n_samples):
    random_i = np.random.choice(len(X), size = n_samples)
    X_trunc = X[random_i]
    y_trunc = y[random_i]
    return X_trunc, y_trunc

In [23]:
int(X.shape[0] * 0.01)

6049

In [24]:
X_truncated, y_truncated = truncate_data(X, y, int(X.shape[0] * 0.01))

In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_truncated, y_truncated, test_size = 0.2)

# analysis of data

In [26]:
X_sample = X[np.random.choice(len(X))]
y_sample = y[np.random.choice(len(y))]
print("random X min:", np.min(X_sample))
print("random X max:", np.max(X_sample))
print("random y min:", np.min(y_sample))
print("random y max:", np.max(y_sample))

random X min: -154.36665
random X max: 659.13336
random y min: -0.33333334
random y max: 0.0


In [27]:
print("x min:", np.min(X))
print("x max:", np.max(X))

x min: -34065332.0
x max: 48234530.0


In [28]:
y[:,6].max()

1.6666666

In [29]:
print("y min:", np.min(y))
print("y max:", np.max(y))
print("y mean:", np.mean(y[:,1:]))
print("y std:", np.std(y))

y min: -3.0
y max: 3.0
y mean: 0.17519538
y std: 0.56088483


In [30]:
for i in range(y.shape[1]):
    print("min y, index", i, ":", np.min(y[:,i]))
    print("max y, index", i, ":", np.max(y[:,i]), "\n")


min y, index 0 : -3.0
max y, index 0 : 3.0 

min y, index 1 : 0.0
max y, index 1 : 3.0 

min y, index 2 : 0.0
max y, index 2 : 3.0 

min y, index 3 : 0.0
max y, index 3 : 3.0 

min y, index 4 : 0.0
max y, index 4 : 3.0 

min y, index 5 : 0.0
max y, index 5 : 3.0 

min y, index 6 : 0.0
max y, index 6 : 1.6666666 



per the paper, the integral [-3, 3] for the target values, as implied above (min/max) represents sentiment, with +3 being highly positive, -3 highly negative, 0 neutral. the mean value is also consistent with the distribution shown in the paper (pp 2240 fig 2), which shows that most annotations are neutral or weakly positive (0-1). do we need to balance the classes at some point? **note that the first index (i=0) of any given target array represents this integral

the paper mentions a second integral [0,3] which measures the presence of emotion (0 = no presence, 3 = highest presence). it looks like indices 1-5 of the target arrays are of this integral.

I'm not quite sure what the values in the final index (values range from [0, 2)) represent in the data. best assumption is that it's the same as the previous 5 values, because there are 6 emotions (happiness, sadness, anger, fear, disgust, surprise)

In [31]:
print("train:", X_train.shape)
print("test:", X_test.shape)

train: (4839, 713)
test: (1210, 713)


In [32]:
X_train[2].shape

(713,)

# model

In [16]:
# import tensorflow as tf

# model = tf.keras.models.Sequential([
#   tf.keras.layers.Flatten(input_shape=(X_train.shape[1],)),
#   tf.keras.layers.Dense(712, activation='relu'),
#   tf.keras.layers.Dense(712, activation='relu'),
#   tf.keras.layers.Dense(356, activation='relu'),
#   tf.keras.layers.Dense(178, activation='relu'),
#   tf.keras.layers.Dense(7, activation='softmax')
# ])

In [17]:
# loss = tf.keras.losses.MeanSquaredError()

In [18]:
# model.compile(optimizer='adam',
#   loss=loss,
#   metrics=['accuracy'])

In [19]:
# model.fit(X_train, y_train, epochs=10)

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier

clf = DecisionTreeClassifier(random_state=0)

In [34]:
from sklearn.multioutput import MultiOutputRegressor
from sklearn.ensemble import GradientBoostingRegressor

clf = MultiOutputRegressor(GradientBoostingRegressor(random_state=0))

In [37]:
trained = clf.fit(X_train,y_train)

In [58]:
trained.predict(X_test)

array([[ 0.20646624,  1.11828705,  0.16974046, ...,  0.06052888,
         0.17677589,  0.04378128],
       [-0.00895974,  0.47875212,  0.20112771, ...,  0.05984263,
         0.20490811,  0.0327238 ],
       [ 0.38816177,  0.69448503,  0.12929643, ...,  0.05930335,
         0.01839818,  0.04337723],
       ...,
       [-0.14418211,  0.21003477,  0.08317705, ...,  0.04908758,
         0.09206759,  0.00165839],
       [ 0.13683652,  0.56096793,  0.12085602, ...,  0.06018559,
         0.17915968,  0.02938102],
       [ 0.35032274,  0.91025027,  0.11037594, ...,  0.02353664,
         0.06498344,  0.03875004]])

In [52]:
y_test

array([[ 2.        ,  1.6666666 ,  0.        , ...,  1.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.33333334,
         0.        ,  0.        ],
       [ 1.3333334 ,  1.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-1.        ,  0.        ,  0.6666667 , ...,  0.        ,
         0.        ,  0.33333334],
       [-2.6666667 ,  0.        ,  0.        , ...,  0.        ,
         2.        ,  0.        ],
       [ 1.6666666 ,  1.6666666 ,  0.        , ...,  0.        ,
         0.        ,  1.        ]], dtype=float32)